# **Analytics 2 :** <font color=#DF4807>**Transformers**</font>

In [4]:
!pip install datasets --quiet
import pandas as pd
from datasets import load_dataset

In [5]:
emotions = load_dataset("SetFit/emotion")
emotions_train = emotions['train']

Repo card metadata block was not found. Setting CardData to empty.


In [6]:
emotions.set_format(type='pandas')
df = emotions['train'][:]
df.head(10)

,text,label,label_text
0,i didnt feel humiliated,0,sadness
1,i can go from feeling so hopeless to so damned...,0,sadness
2,im grabbing a minute to post i feel greedy wrong,3,anger
3,i am ever feeling nostalgic about the fireplac...,2,love
4,i am feeling grouchy,3,anger
5,ive been feeling a little burdened lately wasn...,0,sadness
6,ive been taking or milligrams or times recomme...,5,surprise
7,i feel as confused about life as a teenager or...,4,fear
8,i have been with petronas for years i feel tha...,1,joy
9,i feel romantic too,2,love


In [7]:
df['label_text'].nunique()

6

In [8]:
df['text'][2]

'im grabbing a minute to post i feel greedy wrong'

## **Tasks**

1. Fine tune a classification model using the dataset provided.
2. Test your trained model on unseen data.
3. If you are having difficulties with the multiclass classification, you can tweak the labels into 2 i.e. positive or negative to simplify.

In [9]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [10]:
#install requirements

!pip install transformers datasets torch  -q gwpy
!pip install sentencepiece -q gwpy

from datasets import load_dataset
from datasets import load_metric #for access to metrics like F1 score, precision, recall...

from transformers import pipeline
from transformers import DataCollatorWithPadding #help batch data and pad them to a common lenght

from transformers import AutoTokenizer #generic tokenizer that we can set to match a pre-trained model
from transformers import AutoModelForQuestionAnswering, DistilBertConfig #AutoModel class plus bert config
from transformers import Trainer, TrainingArguments #used for fine tuning a model

from sklearn.metrics import f1_score
import torch

In [11]:

tokenizer = AutoTokenizer.from_pretrained("ayoubkirouane/BERT-Emotions-Classifier", model_max_length=512)

c:\Users\Poorn\PycharmProjects\NLP\Exam Practice\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
df[['label','label_text']]

,label,label_text
0,0,sadness
1,0,sadness
2,3,anger
3,2,love
4,3,anger
...,...,...
15995,0,sadness
15996,0,sadness
15997,1,joy
15998,3,anger


In [14]:
id2label = {0: "sadness",1:'joy', 2: "love",3:'anger',4:'fear',5:'surprise'}

# label2id = {"sadness": 0, "joy": 1,'love':2,'anger':3,'fear':4,'surprise':5}

In [15]:
X = df['text']
y= df['label']

In [16]:
from transformers import AutoModelForSequenceClassification #automodel class provided by HF, allows us to select models for classification tasks
#NOTE: sequence classification tasks include sentiment analysis, text categorization, and natural language inference.
from transformers import TrainingArguments, Trainer #classes that we need for fine tuning

In [17]:
from sklearn.model_selection import train_test_split

# Assuming X contains your features and y contains your labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [18]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [19]:

model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased", id2label=id2label)

c:\Users\Poorn\PycharmProjects\NLP\Exam Practice\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
pip install accelerate -U

Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install transformers[torch]


Note: you may need to restart the kernel to use updated packages.


In [25]:
training_args = TrainingArguments(
    output_dir='./results',           # output directory where the model predictions and checkpoints will be written.
    learning_rate=2e-5,               # learning rate
    evaluation_strategy = "epoch",    # you can either define epoch or steps for your training cycles
    per_device_train_batch_size=16,   # batch size per GPU/TPU core/CPU for training
    per_device_eval_batch_size=16,    # batch size per GPU/TPU core/CPU for evaluation
    num_train_epochs=1,               # total number of training epochs
    weight_decay=0.01,                # strength of weight decay (regularisation technique to reduce overfitting), higher values of weight_decay result in stronger regularization
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=X_train,
    eval_dataset=X_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [24]:
!pip install -U accelerate
!pip install -U transformers